In [265]:
import networkfunctions as nf
import random
import pandas as pd
import numpy as np

In [266]:
random.seed(2)
def generateData(nBytes):
  nBytes -= 1
  start = [[1], [0]]

  for i in range(nBytes):
    new_start = []
    for j in start:
      new_start.append(j + [0])
      new_start.append(j + [1])
    start = new_start

  return start

In [267]:
def do_training_bottomless(net, train_inputs, train_outputs, learning_rate):
    #min = float('inf')  #super large value
    epoch = 1
    breaker = 1
    n = 0


In [268]:
# CB
def minError(nInputs, nHidden, data, runs):
  network = nf.make_network(nInputs, nHidden, nInputs)
  # BK - use np to tell you initial min
  absoluteMin = np.finfo(np.float32).max;
  for i in range(runs):
    random.seed(i)
    #min = do_training_bottomless(network, data, data, 0.5)
    min = nf.do_training(network, data, data, 0.5, 10000)
    if min < absoluteMin:
      absoluteMin = min
  return absoluteMin

In [269]:
def experimentVaryingInput(maxInput, fixedHidden):
  IV = []
  DV = []
  runs = 20
  i = 1
  for inputs in range(1, maxInput):
    print("Working on inputs="+str(inputs))
    data = generateData(inputs)
    error = minError(inputs, fixedHidden, data, runs)
    IV.append(inputs)
    DV.append(error)
    i+=1
  return IV, DV

In [270]:
def experimentVaryingHidden(maxHidden, fixedInput):
    IV = []
    DV = []
    runs = 20
    i = 1
    for hidden in range(1, maxHidden):
        print("Working on hidden="+str(hidden))
        data = generateData(fixedInput)
        error = minError(fixedInput, hidden, data, runs)
        IV.append(hidden)
        DV.append(error)
        i+=1
    return IV, DV

In [271]:
def generateResultsDf(N, M, numEpochs): #N = INPUT NODES    N = HIDDEN NODES
    inputNodes = []
    hiddenNodes = []
    epochHolder = []
    
    
    N += 1
    M += 1
    for i in range(1, N):
        for j in range(1, M):
            for k in range(0, numEpochs, 1000):
                inputNodes.append(i)
                hiddenNodes.append(j)
                epochHolder.append(k + 1000)
            
    blank = []

    data = {
        'N': inputNodes,
        'M': hiddenNodes,
        'epoch': epochHolder,
        'error': None
    }
    df = pd.DataFrame(data)
    df.to_csv('results.csv', encoding='utf-8', index=False)
    return df

In [272]:
def generateJobsDf(N, M): #N = INPUT NODES    N = HIDDEN NODES
    inputNodes = []
    hiddenNodes = []
    epochHolder = []
    
    N += 1
    M += 1
    for i in range(1, N):
        for j in range(1, M):
            inputNodes.append(i)
            hiddenNodes.append(j)
            epochHolder.append(10000)
            
    blank = []

    data = {
        'N': inputNodes,
        'M': hiddenNodes,
        'epoch': epochHolder,
        'error': None
    }
    df = pd.DataFrame(data)
    df.to_csv('jobQ.csv', encoding='utf-8', index=False)
    return df

In [273]:
generateResultsDf(10,10, 10000)

,N,M,epoch,error
0,1,1,1000,None
1,1,1,2000,None
2,1,1,3000,None
3,1,1,4000,None
4,1,1,5000,None
...,...,...,...,...
995,10,10,6000,None
996,10,10,7000,None
997,10,10,8000,None
998,10,10,9000,None


In [274]:
generateJobsDf(10,10)

,N,M,epoch,error
0,1,1,10000,None
1,1,2,10000,None
2,1,3,10000,None
3,1,4,10000,None
4,1,5,10000,None
...,...,...,...,...
95,10,6,10000,None
96,10,7,10000,None
97,10,8,10000,None
98,10,9,10000,None


In [275]:
jobQ = pd.read_csv('jobQ.csv')
results = pd.read_csv('results.csv')

In [276]:
#### MAIN
##not updating properly every time for loop runs again
# generateResultsDf(10,10, 10000)
# generateJobsDf(10,10)

# jobQ = pd.read_csv('jobQ.csv')
# results = pd.read_csv('results.csv')

for index, row in results.iterrows():
    error = []
    if pd.isna(row["error"]):
        net = nf.make_network(int(row["N"]), int(row["M"]), int(row["N"]))
        data = generateData(int(row["N"]))
        for i in range(10):
            min_error = nf.do_training(net, data, data, 0.5, (i+1)*1000)
            error.append(min_error)
        for i in range(10):
            results.loc[index+i, "error"] = error[i]
        jobIndex = jobQ[(jobQ['N'] == int(row["N"])) & (jobQ['M'] == int(row["M"]))]
        jobQ.loc[jobIndex.index, "error"] = error[9]
        jobQ.to_csv('jobQ.csv', index=False)
        results.to_csv('results.csv', index=False)
        print(error)

[0.0033868671131935895668, 0.0010136262616132495486, 0.00048713110067298461504, 0.0002860500479069351853, 0.00018808741234681010194, 0.0001330536840598024418, 9.9073368617225314556e-05, 7.6626917878696434697e-05, 6.1027311857652671103e-05, 4.9748104473855167238e-05]
[0.004256283310872289434, 0.0012812205570649301002, 0.00061340246281393234177, 0.0003590637301278652055, 0.00023555464313491922696, 0.00016635072968285502288, 0.00012370845754541168195, 9.558553061825957143e-05, 7.606614636589997758e-05, 6.1967580438475349615e-05]
[0.008182712373147248652, 0.0023673165101668713715, 0.0011131758589674271618, 0.0006452560645320302098, 0.0004207333270450674928, 0.00029590177578670871238, 0.00021939760141492388525, 0.00016914272179592494124, 0.00013436806465203773008, 0.0001093110149185378764]
[0.0177448869782502845, 0.0048260022829181370895, 0.002213136933682507576, 0.0012663332999764263194, 0.00081925855553309756443, 0.0005731811790392037997, 0.00042340545197139162803, 0.000325510098098366110